In [1]:
from roboflow import Roboflow
rf = Roboflow(api_key="S3eK0rrtbGBZFhsHBOEK")
project = rf.workspace("autoannotate-study").project("berries-n34za")
version = project.version(1)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.49, to fix: `pip install ultralytics==8.0.196`


In [ ]:
rf = Roboflow(api_key="S3eK0rrtbGBZFhsHBOEK")
project = rf.workspace("autoannotate-study").project("fescue")
version = project.version(1)
dataset = version.download("yolov8")


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="S3eK0rrtbGBZFhsHBOEK")
project = rf.workspace("autoannotate-study").project("red-leaf")
version = project.version(1)
dataset = version.download("yolov8")

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="S3eK0rrtbGBZFhsHBOEK")
project = rf.workspace("autoannotate-study").project("buds-79ct6")
version = project.version(1)
dataset = version.download("yolov8")

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="8t6x218rPQnJEC3RbLZq")
project = rf.workspace("auto-annotate-2").project("tiled-fescue-boxes")
version = project.version(2)
dataset = version.download("yolov8")

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="8t6x218rPQnJEC3RbLZq")
project = rf.workspace("auto-annotate-2").project("tiled-fescue-seg")
version = project.version(1)
dataset = version.download("yolov8")

In [2]:
import time
from pathlib import Path
import torch
from ultralytics import SAM, YOLO

def clean_labels(boxes, max_area):
    clean_boxes = []
    box_list = boxes.tolist()
    for box in box_list:
        if ((box[3]-box[1])*(box[2]-box[0]))<max_area:
            clean_boxes.append(box)
    if len(clean_boxes)<1:
        return boxes
    torch_tens = torch.FloatTensor(clean_boxes)
    return torch_tens

def auto_annotate(data, det_model="yolov8x.pt", sam_model="sam_b.pt", device="", output_dir=None, prompt="Bud on shoot", confidence=0.1, box_threshold=0.2):
    """
    Automatically annotates images using a YOLO object detection model and a SAM segmentation model.

    Args:
        data (str): Path to a folder containing images to be annotated.
        det_model (str, optional): Pre-trained YOLO detection model. Defaults to 'yolov8x.pt'.
        sam_model (str, optional): Pre-trained SAM segmentation model. Defaults to 'sam_b.pt'.
        device (str, optional): Device to run the models on. Defaults to an empty string (CPU or GPU, if available).
        output_dir (str | None | optional): Directory to save the annotated results.
            Defaults to a 'labels' folder in the same directory as 'data'.

    Example:
        ```python
        from ultralytics.data.annotator import auto_annotate

        auto_annotate(data='ultralytics/assets', det_model='yolov8n.pt', sam_model='mobile_sam.pt')
        ```
    """

    if 'world' in det_model:
        det_model = YOLO(det_model)
        det_model.set_classes([prompt])
    else:
        det_model = YOLO(det_model)
    sam_model = SAM(sam_model)

    data = Path(data)
    if not output_dir:
        output_dir = data.parent / f"{data.stem}_auto_annotate_labels"
    Path(output_dir).mkdir(exist_ok=True, parents=True)


    #det_results = det_model(data, stream=True, device=device)
    det_results = det_model.predict(data,conf=confidence,save=True, save_txt=True, verbose=False)

    for result in det_results:
        class_ids = result.boxes.cls.int().tolist()  # noqa
        shape = result.orig_shape
        boxes = result.boxes.xyxy
        max_area = shape[0]*shape[1]*box_threshold
        boxes = clean_labels(boxes, max_area)
        if len(boxes):
            sam_results = sam_model(result.orig_img, bboxes=boxes, verbose=False, save=True, device=device)
            segments = sam_results[0].masks.xyn  # noqa

            with open(f"{Path(output_dir) / Path(result.path).stem}.txt", "w") as f:
                for i in range(len(segments)):
                    s = segments[i]
                    if len(s) == 0:
                        continue
                    segment = map(str, segments[i].reshape(-1).tolist())
                    f.write(f"{class_ids[i]} " + " ".join(segment) + "\n")

# Evaluation

In [3]:
from PIL import Image, ImageDraw
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt

def calculate_metrics(TP, FP, FN, TN):
    precision = TP / (TP + FP) if TP + FP > 0 else 0
    recall = TP / (TP + FN) if TP + FN > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
    mcc = ((TP * TN) - (FP * FN)) / np.sqrt((TP+FP) * (TP+FN) * (TN+FP) * (TN+FN)) if np.sqrt((TP+FP) * (TP+FN) * (TN+FP) * (TN+FN)) > 0 else 0
    specificity = TN / (TN + FP) if TN + FP > 0 else 0
    return precision, recall, f1, mcc, specificity

def pixel_accuracy(predicted, ground_truth):
    correct = np.sum(predicted == ground_truth)
    total = predicted.shape[0] * predicted.shape[1]
    return correct / total

def read_and_draw_masks(file_path, image_dim=(1280, 720)):

    with open(file_path, 'r') as file:
        masks = []
        for line in file:
            raw_mask = [float(x) for x in list(line.strip().split())]
            points = []
            for point in range(int((len(raw_mask)-1)/2)):
                p1 = int(raw_mask[(2*point)+1]*image_dim[0])
                p2 = int(raw_mask[(2*point)+2]*image_dim[1])
                points.append([p1,p2])
            masks.append(points)
    canvas = np.zeros((image_dim[1], image_dim[0]), dtype=np.uint8)
    for mask in masks:
        cv2.fillPoly(canvas, np.array([mask], dtype=np.int32), 255)
    return np.array(canvas, dtype=np.uint8)

def calculate_pixel_metrics(mask1, mask2):
    """
    Calculate IoU based on pixel values from two masks.
    """
    intersection = np.logical_and(mask1, mask2).sum()
    union = np.logical_or(mask1, mask2).sum()
    if union == 0:
        return 0
    return intersection / union

def process_files_seg(predicted_mask_dir, ground_truth_mask_dir):
    predicted_files = os.listdir(ground_truth_mask_dir)
    metrics = {
        'iou_scores': [],
        'pixel_accuracies': [],
        'precision_scores': [],
        'recall_scores': [],
        'f1_scores': [],
        'mcc_scores': [],
        'specificity_scores': []
    }

    for fname in predicted_files:
        predicted_mask_path = os.path.join(predicted_mask_dir, fname)
        ground_truth_mask_path = os.path.join(ground_truth_mask_dir, os.path.splitext(fname)[0] + '.txt')
        if not os.path.exists(predicted_mask_path):
            metrics['iou_scores'].append(0)
            metrics['pixel_accuracies'].append(0)
            metrics['precision_scores'].append(0)
            metrics['recall_scores'].append(0)
            metrics['f1_scores'].append(0)
            metrics['mcc_scores'].append(0)
            metrics['specificity_scores'].append(0)
            continue

        COMMON_HEIGHT, COMMON_WIDTH = 1280, 720  # or any other desired size

        predicted_mask = read_and_draw_masks(predicted_mask_path)

        ground_truth_mask = read_and_draw_masks(ground_truth_mask_path)

        predicted_mask = cv2.resize(predicted_mask, (COMMON_WIDTH, COMMON_HEIGHT))
        ground_truth_mask = cv2.resize(ground_truth_mask, (COMMON_WIDTH, COMMON_HEIGHT))

        _, predicted_mask_bin = cv2.threshold(predicted_mask, 127, 255, cv2.THRESH_BINARY)
        _, ground_truth_mask_bin = cv2.threshold(ground_truth_mask, 127, 255, cv2.THRESH_BINARY)

        predicted_mask_bin = predicted_mask_bin / 255
        ground_truth_mask_bin = ground_truth_mask_bin / 255
        TP = np.float64(np.sum(np.logical_and(predicted_mask_bin == 1, ground_truth_mask_bin == 1)))
        TN = np.float64(np.sum(np.logical_and(predicted_mask_bin == 0, ground_truth_mask_bin == 0)))
        FP = np.float64(np.sum(np.logical_and(predicted_mask_bin == 1, ground_truth_mask_bin == 0)))
        FN = np.float64(np.sum(np.logical_and(predicted_mask_bin == 0, ground_truth_mask_bin == 1)))


        intersection = np.logical_and(predicted_mask_bin, ground_truth_mask_bin)
        union = np.logical_or(predicted_mask_bin, ground_truth_mask_bin)
        metrics['iou_scores'].append(np.sum(intersection) / np.sum(union))
        metrics['pixel_accuracies'].append(pixel_accuracy(predicted_mask_bin, ground_truth_mask_bin))
        precision, recall, f1, mcc, specificity = calculate_metrics(TP, FP, FN, TN)
        metrics['precision_scores'].append(precision)
        metrics['recall_scores'].append(recall)
        metrics['f1_scores'].append(f1)
        metrics['mcc_scores'].append(mcc)
        metrics['specificity_scores'].append(specificity)

    return metrics

# Prompt Optimization

In [9]:
def run_yolo_world(image_path, prompt, conf, model_size, save_image = False, save_label= True):

    model = YOLO(model_size)  # or choose yolov8m/l-world.pt

    model.set_classes([prompt])

    results = model.predict(image_path,conf=conf, save=save_image, save_txt=save_label, verbose=True)
    print(results[0].boxes.xyxy.cpu().tolist())
    return results

In [10]:
import time as t
def optimize_confidence(prompt, model_size, gt_path, img_dir, threshold):
    best_iou = 0
    best_conf = 0
    #number of decimal points in confidence
    final_precision = 5
    ubound = 0.1
    lbound = 0.9
    for precision in [x for x in range(final_precision)]:
        for conf in [x / (10 ** precision) for x in range(int(lbound * (10 ** precision)), int(ubound * (10 ** precision)))]:
            results = run_yolo_world(img_dir, prompt, conf, model_size)
            metrics = process_files(results[0].save_dir+r"/labels/", gt_path, thresh=threshold)
            #print(metrics)
            iou = np.mean(metrics['iou_scores'])
            if iou > best_iou:
                best_iou = iou
                best_conf = conf
            print(f"confidence: {conf}, IOU: {iou} (best: {best_iou})")
        print(f"Best IOU at p{precision} is {best_iou} with confidence = {best_conf}")
        lbound = max(0, best_conf - (1 / (10 ** precision)))
        ubound = min(0.9, best_conf + (1 / (10 ** precision)))
        if (best_conf > (0.2*(10**precision))) and precision >=2:
            print(f"Final Result: Best IOU  is {best_iou} with confidence = {best_conf}")
            return best_iou, best_conf

    return best_iou, best_conf

def multi_optmize(img_dir, gt_label_dir, model_size, prompts, threshold=0.8):
    print("Be sure to change the category folders and model size in each function!")
    t.sleep(2)
    start = t.time()
    best_iou = 0
    for prompt in prompts:
        print(f"Trying prompt: '{prompt}'")
        iou, conf = optimize_confidence(prompt, model_size, gt_label_dir, img_dir, threshold=threshold)
        if iou > best_iou:
            best_iou = iou
            best_conf = conf
            best_prompt = prompt
        print(f"So far: best prompt is '{best_prompt}', conf is {best_conf}, resulting in {best_iou} IOU)")
    print(f"\n\n\n\n\nFinal Result: best prompt is '{best_prompt}', conf is {best_conf}, resulting in {best_iou} IOU)")
    print(f"final time: {t.time() - start}")
    return {"prompt": best_prompt, "conf": best_conf, "iou": best_iou}

In [11]:
def optimize_prompts(prompts_file, gt_path, img_dir, save_file, confidence=0.1, threshold=0.2):

    with  open(prompts_file, 'r') as file:
        result_dict = {}
        for x in file:
            result_dict[x.strip()] = {}

    #result_dict = dict.fromkeys(prompts,{})
    for prompt in result_dict.keys():
        print(f'Trying prompt: "{prompt}"')
        model_size = 'yolov8l-worldv2.pt'
        results = run_yolo_world(img_dir, prompt, confidence, model_size)
        inf_path= results[0].save_dir+r"/labels/"
        metrics = process_files(inf_path, gt_path, thresh=threshold)
        result_dict[prompt]['iou_scores'] = np.mean(metrics['iou_scores'])
        print(f"IOU for {prompt}: {result_dict[prompt]['iou_scores']}")

    results = sorted(list(result_dict.items()), key=lambda a:a[1]['iou_scores'], reverse=True)
    return results

In [12]:
from PIL import Image, ImageDraw
import numpy as np

def calculate_metrics(TP, FP, FN, TN):
    precision = TP / (TP + FP) if TP + FP > 0 else 0
    recall = TP / (TP + FN) if TP + FN > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
    mcc = ((TP * TN) - (FP * FN)) / np.sqrt((TP+FP) * (TP+FN) * (TN+FP) * (TN+FN)) if np.sqrt((TP+FP) * (TP+FN) * (TN+FP) * (TN+FN)) > 0 else 0
    specificity = TN / (TN + FP) if TN + FP > 0 else 0
    return precision, recall, f1, mcc, specificity
def pixel_accuracy(predicted, ground_truth):
    correct = np.sum(predicted == ground_truth)
    total = predicted.shape[0] * predicted.shape[1]
    return correct / total

def clean_labels_from_file(file_path, cleaning_threshold=0.6):
    # Read the file and check if it has more than one line
    with open(file_path, 'r') as f:
        lines = f.readlines()

    if len(lines) > 1:
        accepted_lines = []

        # Process each line
        for line in lines:
            class_id, x, y, width, height = map(float, line.strip().split())
            #if width * height < 0.9:
            if (width*height)<cleaning_threshold:
                accepted_lines.append(line)

        # Overwrite the file with accepted lines
        with open(file_path, 'w') as f:
            if len(accepted_lines)>0:
                for line in accepted_lines:
                    f.write(line)


#Uses absolute boxes (NOT NORMALIZED)
def read_and_draw_boxes(file_path, image_dim=(1280, 720)):
    boxes = []
    with open(file_path, 'r') as file:
        for line in file:
            class_id, x, y, width, height = map(float, line.strip().split())
            x1 = (x-(width/2))*image_dim[0]
            x2 = (x+(width/2))*image_dim[0]
            y1 = (y-(height/2))*image_dim[1]
            y2 = (y+(height/2))*image_dim[1]
            boxes.append([x1, y1, x2, y2])

    image = Image.new('L', image_dim, 0)
    draw = ImageDraw.Draw(image)
    for box in boxes:
        draw.rectangle(box, fill=255)
        #draw.rectangle([1,1,20,20], fill=255)
    image.save("test.jpg")
    return np.array(image, dtype=np.uint8)

def calculate_pixel_metrics(mask1, mask2):
    """
    Calculate IoU based on pixel values from two masks.
    """
    intersection = np.logical_and(mask1, mask2).sum()
    union = np.logical_or(mask1, mask2).sum()
    if union == 0:
        return 0
    return intersection / union

def process_files(predicted_mask_dir, ground_truth_mask_dir, thresh):
    predicted_files = os.listdir(ground_truth_mask_dir)
    metrics = {
        'iou_scores': [],
        'pixel_accuracies': [],
        'precision_scores': [],
        'recall_scores': [],
        'f1_scores': [],
        'mcc_scores': [],
        'specificity_scores': []
    }

    for fname in predicted_files:
        predicted_mask_path = os.path.join(predicted_mask_dir, fname)
        ground_truth_mask_path = os.path.join(ground_truth_mask_dir, os.path.splitext(fname)[0] + '.txt')
        if not os.path.exists(predicted_mask_path):
            metrics['iou_scores'].append(0)
            metrics['pixel_accuracies'].append(0)
            metrics['precision_scores'].append(0)
            metrics['recall_scores'].append(0)
            metrics['f1_scores'].append(0)
            metrics['mcc_scores'].append(0)
            metrics['specificity_scores'].append(0)
            continue
        clean_labels_from_file(predicted_mask_path, cleaning_threshold=thresh)
        predicted_mask = read_and_draw_boxes(predicted_mask_path)
        ground_truth_mask = read_and_draw_boxes(ground_truth_mask_path)

        COMMON_HEIGHT, COMMON_WIDTH = 1280, 720  # or any other desired size

        predicted_mask = cv2.resize(predicted_mask, (COMMON_WIDTH, COMMON_HEIGHT))

        ground_truth_mask = cv2.resize(ground_truth_mask, (COMMON_WIDTH, COMMON_HEIGHT))

        _, predicted_mask_bin = cv2.threshold(predicted_mask, 127, 255, cv2.THRESH_BINARY)
        _, ground_truth_mask_bin = cv2.threshold(ground_truth_mask, 127, 255, cv2.THRESH_BINARY)

        predicted_mask_bin = predicted_mask_bin / 255
        ground_truth_mask_bin = ground_truth_mask_bin / 255
        TP = np.float64(np.sum(np.logical_and(predicted_mask_bin == 1, ground_truth_mask_bin == 1)))
        TN = np.float64(np.sum(np.logical_and(predicted_mask_bin == 0, ground_truth_mask_bin == 0)))
        FP = np.float64(np.sum(np.logical_and(predicted_mask_bin == 1, ground_truth_mask_bin == 0)))
        FN = np.float64(np.sum(np.logical_and(predicted_mask_bin == 0, ground_truth_mask_bin == 1)))


        intersection = np.logical_and(predicted_mask_bin, ground_truth_mask_bin)
        union = np.logical_or(predicted_mask_bin, ground_truth_mask_bin)
        IoU = calculate_pixel_metrics(predicted_mask_bin, ground_truth_mask_bin)
        #metrics['iou_scores'].append(np.sum(intersection) / np.sum(union))
        metrics['iou_scores'].append(IoU)
        metrics['pixel_accuracies'].append(pixel_accuracy(predicted_mask_bin, ground_truth_mask_bin))
        precision, recall, f1, mcc, specificity = calculate_metrics(TP, FP, FN, TN)
        metrics['precision_scores'].append(precision)
        metrics['recall_scores'].append(recall)
        metrics['f1_scores'].append(f1)
        metrics['mcc_scores'].append(mcc)
        metrics['specificity_scores'].append(specificity)

    return metrics

# Berries

In [13]:
prompts_file = r'C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\blueberry-prompts.txt'
ground_truth_dir = r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\bounding-berries\train\labels"

results = optimize_prompts(prompts_file,ground_truth_dir, r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\bounding-berries\train\images", 'berry-results-dino.txt')

top10 = [result[0] for result in results][0:10]

Trying prompt: "blueberry"

image 1/6 C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\bounding-berries\train\images\IMG_9331_jpg.rf.20009327b80c55eec840b8b4f5cddf57.jpg: 448x640 1 blueberry, 27.1ms
image 2/6 C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\bounding-berries\train\images\IMG_9355_jpg.rf.40d4de298491188a33bcdfd995d9e855.jpg: 448x640 1 blueberry, 26.0ms
image 3/6 C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\bounding-berries\train\images\IMG_9379_jpg.rf.42c280b08420d4271486e3cdebe8a30e.jpg: 448x640 1 blueberry, 27.0ms
image 4/6 C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\bounding-berries\train\images\IMG_9383_jpg.rf.7af81e391f70df26bca8c741d75bcf24.jpg: 448x640 1 blueberry, 25.3ms
image 5/6 C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\bounding-berries\train\images\IMG_9387_jpg.rf.9ae726fc1ddc490013a19db8c1c2a1f1.jpg: 448x640 (no det

KeyboardInterrupt: 

In [ ]:
multi_optmize(r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\bounding-berries\train\images", r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\bounding-berries\train\labels", 'yolov8s-worldv2.pt', top10)

In [ ]:
multi_optmize(r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\bounding-berries\train\images", r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\bounding-berries\train\labels", 'yolov8m-worldv2.pt', top10)

In [ ]:
multi_optmize(r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\bounding-berries\train\images", r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\bounding-berries\train\labels", 'yolov8l-worldv2.pt', top10)

In [ ]:
multi_optmize(r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\bounding-berries\train\images", r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\bounding-berries\train\labels", 'yolov8x-worldv2.pt', top10)

# RedLeaf

In [ ]:
prompts_file = r'C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\red_leaf_plant_prompts.txt'
ground_truth_dir = r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\red-leaf-bounding-box-1\train\labels"

results = optimize_prompts(prompts_file,ground_truth_dir, r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\red-leaf-bounding-box-1\train\images", 'redleaf-results-dino.txt')

top10 = [result[0] for result in results][0:10]

In [ ]:
multi_optmize(r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\red-leaf-bounding-box-1\train\images", r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\red-leaf-bounding-box-1\train\labels", 'yolov8s-worldv2.pt', top10)

In [ ]:
multi_optmize(r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\red-leaf-bounding-box-1\train\images", r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\red-leaf-bounding-box-1\train\labels", 'yolov8m-worldv2.pt', top10)

In [ ]:
multi_optmize(r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\red-leaf-bounding-box-1\train\images", r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\red-leaf-bounding-box-1\train\labels", 'yolov8l-worldv2.pt', top10)

In [ ]:
multi_optmize(r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\red-leaf-bounding-box-1\train\images", r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\red-leaf-bounding-box-1\train\labels", 'yolov8x-worldv2.pt', top10)

 # Fescue

In [26]:
prompts_file = r'C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\Fescue_prompts.txt'
ground_truth_dir = r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\tiled-fescue-boxes-2\train\labels"

results = optimize_prompts(prompts_file,ground_truth_dir, r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\tiled-fescue-boxes-2\train\images", 'fescue-results-dino.txt', confidence=0.0005, threshold=0.5)

top10 = [result[0] for result in results][0:10]

Trying prompt: "Fescue grass patches"
Results saved to runs\detect\predict66
7 labels saved to runs\detect\predict66\labels
IOU for Fescue grass patches: 0.1553890448672249
Trying prompt: "Fescue grass areas"
Results saved to runs\detect\predict67
5 labels saved to runs\detect\predict67\labels
IOU for Fescue grass areas: 0.0747646118358501
Trying prompt: "Fescue grass clusters"
Results saved to runs\detect\predict68
9 labels saved to runs\detect\predict68\labels
IOU for Fescue grass clusters: 0.1924891016245326
Trying prompt: "Fescue grass regions"
Results saved to runs\detect\predict69
6 labels saved to runs\detect\predict69\labels
IOU for Fescue grass regions: 0.0347463255479278
Trying prompt: "Fescue grass spots"
Results saved to runs\detect\predict70
11 labels saved to runs\detect\predict70\labels
IOU for Fescue grass spots: 0.23923500837306155
Trying prompt: "Grass patches"
Results saved to runs\detect\predict71
0 label saved to runs\detect\predict71\labels
IOU for Grass patches: 

In [27]:
print(results)

[('Areas of fescue', {'iou_scores': 0.2704751183154504}), ('Fescue patches', {'iou_scores': 0.26463154352528256}), ('Areas of fescue grass', {'iou_scores': 0.25812796452696735}), ('Areas of grass', {'iou_scores': 0.24911555053102352}), ('Fescue regions', {'iou_scores': 0.23938691850870633}), ('Fescue grass spots', {'iou_scores': 0.23923500837306155}), ('Regions of fescue grass', {'iou_scores': 0.2227551517099284}), ('Clusters of grass', {'iou_scores': 0.2226838639432642}), ('Fescue grass clusters', {'iou_scores': 0.1924891016245326}), ('Clusters', {'iou_scores': 0.1836971866917663}), ('Clusters of fescue grass', {'iou_scores': 0.1822962557986875}), ('Clusters of fescue', {'iou_scores': 0.18223724022151994}), ('Regions of grass', {'iou_scores': 0.16967640593559397}), ('Regions of fescue', {'iou_scores': 0.15889581496733668}), ('Fescue clusters', {'iou_scores': 0.15662752668530977}), ('Fescue grass patches', {'iou_scores': 0.1553890448672249}), ('Patches of fescue', {'iou_scores': 0.1361

In [28]:
multi_optmize(
    r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\tiled-fescue-boxes-2\train\images",
    r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\tiled-fescue-boxes-2\train\labels",
    'yolov8s-worldv2.pt', top10, threshold=0.5)

Be sure to change the category folders and model size in each function!
Trying prompt: 'Areas of fescue'
Best IOU at p0 is 0 with confidence = 0
Results saved to runs\detect\predict104
11 labels saved to runs\detect\predict104\labels
confidence: 0.0, IOU: 0.22240919944359316 (best: 0.22240919944359316)
Results saved to runs\detect\predict105
0 label saved to runs\detect\predict105\labels
confidence: 0.1, IOU: 0.0 (best: 0.22240919944359316)
Results saved to runs\detect\predict106
0 label saved to runs\detect\predict106\labels
confidence: 0.2, IOU: 0.0 (best: 0.22240919944359316)
Results saved to runs\detect\predict107
0 label saved to runs\detect\predict107\labels
confidence: 0.3, IOU: 0.0 (best: 0.22240919944359316)
Results saved to runs\detect\predict108
0 label saved to runs\detect\predict108\labels
confidence: 0.4, IOU: 0.0 (best: 0.22240919944359316)
Results saved to runs\detect\predict109
0 label saved to runs\detect\predict109\labels
confidence: 0.5, IOU: 0.0 (best: 0.2224091994

{'prompt': 'Fescue grass spots', 'conf': 0.0001, 'iou': 0.2932178153378808}

In [29]:
multi_optmize(
    r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\tiled-fescue-boxes-2\train\images",
    r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\tiled-fescue-boxes-2\train\labels",
    'yolov8m-worldv2.pt', top10, threshold=0.5)

Be sure to change the category folders and model size in each function!
Trying prompt: 'Areas of fescue'
Best IOU at p0 is 0 with confidence = 0
Results saved to runs\detect\predict494
11 labels saved to runs\detect\predict494\labels
confidence: 0.0, IOU: 0.22240836095328287 (best: 0.22240836095328287)
Results saved to runs\detect\predict495
0 label saved to runs\detect\predict495\labels
confidence: 0.1, IOU: 0.0 (best: 0.22240836095328287)
Results saved to runs\detect\predict496
0 label saved to runs\detect\predict496\labels
confidence: 0.2, IOU: 0.0 (best: 0.22240836095328287)
Results saved to runs\detect\predict497
0 label saved to runs\detect\predict497\labels
confidence: 0.3, IOU: 0.0 (best: 0.22240836095328287)
Results saved to runs\detect\predict498
0 label saved to runs\detect\predict498\labels
confidence: 0.4, IOU: 0.0 (best: 0.22240836095328287)
Results saved to runs\detect\predict499
0 label saved to runs\detect\predict499\labels
confidence: 0.5, IOU: 0.0 (best: 0.2224083609

{'prompt': 'Areas of grass', 'conf': 0.001, 'iou': 0.22293141748856382}

In [30]:
multi_optmize(
    r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\tiled-fescue-boxes-2\train\images",
    r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\tiled-fescue-boxes-2\train\labels",
    'yolov8l-worldv2.pt', top10, threshold=0.5)

Be sure to change the category folders and model size in each function!
Trying prompt: 'Areas of fescue'
Best IOU at p0 is 0 with confidence = 0
Results saved to runs\detect\predict894
11 labels saved to runs\detect\predict894\labels
confidence: 0.0, IOU: 0.22240836095328287 (best: 0.22240836095328287)
Results saved to runs\detect\predict895
0 label saved to runs\detect\predict895\labels
confidence: 0.1, IOU: 0.0 (best: 0.22240836095328287)
Results saved to runs\detect\predict896
0 label saved to runs\detect\predict896\labels
confidence: 0.2, IOU: 0.0 (best: 0.22240836095328287)
Results saved to runs\detect\predict897
0 label saved to runs\detect\predict897\labels
confidence: 0.3, IOU: 0.0 (best: 0.22240836095328287)
Results saved to runs\detect\predict898
0 label saved to runs\detect\predict898\labels
confidence: 0.4, IOU: 0.0 (best: 0.22240836095328287)
Results saved to runs\detect\predict899
0 label saved to runs\detect\predict899\labels
confidence: 0.5, IOU: 0.0 (best: 0.2224083609

{'prompt': 'Fescue patches', 'conf': 0.0006, 'iou': 0.28946903705607935}

In [31]:
multi_optmize(
    r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\tiled-fescue-boxes-2\train\images",
    r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\tiled-fescue-boxes-2\train\labels",
    'yolov8x-worldv2.pt', top10, threshold=0.5)

Be sure to change the category folders and model size in each function!
Trying prompt: 'Areas of fescue'
Best IOU at p0 is 0 with confidence = 0
Results saved to runs\detect\predict1304
11 labels saved to runs\detect\predict1304\labels
confidence: 0.0, IOU: 0.22240836095328287 (best: 0.22240836095328287)
Results saved to runs\detect\predict1305
0 label saved to runs\detect\predict1305\labels
confidence: 0.1, IOU: 0.0 (best: 0.22240836095328287)
Results saved to runs\detect\predict1306
0 label saved to runs\detect\predict1306\labels
confidence: 0.2, IOU: 0.0 (best: 0.22240836095328287)
Results saved to runs\detect\predict1307
0 label saved to runs\detect\predict1307\labels
confidence: 0.3, IOU: 0.0 (best: 0.22240836095328287)
Results saved to runs\detect\predict1308
0 label saved to runs\detect\predict1308\labels
confidence: 0.4, IOU: 0.0 (best: 0.22240836095328287)
Results saved to runs\detect\predict1309
0 label saved to runs\detect\predict1309\labels
confidence: 0.5, IOU: 0.0 (best: 

{'prompt': 'Clusters of grass', 'conf': 0.0004, 'iou': 0.24735572953159216}

 # Buds

In [ ]:
prompts_file = r'C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\bud_prompts.txt'
ground_truth_dir = r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\bounding-buds-1\train\labels"

results = optimize_prompts(prompts_file,ground_truth_dir, r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\bounding-buds-1\train\images", 'bud-results-dino.txt')

top10 = [result[0] for result in results][0:10]

In [ ]:
multi_optmize(r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\bounding-buds-1\train\images",
    r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\bounding-buds-1\train\labels",
    'yolov8s-worldv2.pt', top10)

In [ ]:
multi_optmize(r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\bounding-buds-1\train\images",
              r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\bounding-buds-1\train\labels",
              'yolov8m-worldv2.pt', top10)

In [ ]:
multi_optmize(r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\bounding-buds-1\train\images",
              r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\bounding-buds-1\train\labels",
              'yolov8l-worldv2.pt', top10)

In [ ]:
multi_optmize(r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\bounding-buds-1\train\images",
              r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\bounding-buds-1\train\labels",
              'yolov8x-worldv2.pt', top10)

# AutoAnnotate With SAM

In [ ]:
paths=[r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\berries-1\train\images", r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\red-leaf-1\train\images", r"C:\Users\Mechanized Systems\DataspellProjects\AutoAnnotate\autoannotate study\tiled-fescue-seg-1\train\images", r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\buds-1\train\images"]

In [ ]:
YW = ['yolov8s-worldv2.pt', 'yolov8m-worldv2.pt', 'yolov8l-worldv2.pt', 'yolov8x-worldv2.pt']
prompt = [
    ['a blue fruit','spherical blueberry','a single, round blueberry','a blue fruit'],
    ['Bright red leaves contrasting with surrounding green leaves','a cluster of red leaves among green plants','a cluster of red leaves among green vegetation','A cluster of red leaves surrounded by green foliage'],
    ['Fescue grass spots','Regions of fescue grass','Fescue patches','Clusters of grass'],
    ['Bud with leaves','Bud with leaves','Bud with sprouting leaves','Bud with leaves']
]
conf = [[0.008,0.0012,0.0001,0.0115],[0.037,0.01,0.0001,0.01],[0.0004,0.0001,0.0006,0.0023],[0.0012,0.007,0.0002,0.0028]]
SAM_model = ['mobile_sam.pt','sam_b.pt','sam_l.pt']

In [1]:
bertimes = []
rltimes = []
budtimes = []
festimes = []

## Mobile SAM

In [ ]:
print("YW with mobile on berries")
for i in range(4):
    start = time.time()
    auto_annotate(
        data=paths[0],
        det_model=YW[i],
        sam_model=SAM_model[0],
        device="cuda",
        output_dir=fr"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\Worldly_SAM\trials\berries\YW_Mobile\{YW[i]}",
        prompt=prompt[0][i],
        confidence=conf[0][i],
        box_threshold=.4
    )
    total = time.time()-start
    print(f"\n\nTime for {YW[i]}: {total}")
    bertimes.append(total)

In [ ]:
print("\n YW with mobile on red leaf")
for i in range(4):
    start = time.time()
    auto_annotate(
        data=paths[1],
        det_model=YW[i],
        sam_model=SAM_model[0],
        device="cuda",
        output_dir=fr"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\Worldly_SAM\trials\red leaf\YW_Mobile\{YW[i]}",
        prompt=prompt[1][i],
        confidence=conf[1][i],
        box_threshold=.4
    )
    total = time.time()-start
    print(f"\n\nTime for {YW[i]}: {total}")
    rltimes.append(total)

In [34]:
print("\n YW with mobile on fescue")
for i in range(4):
    start = t.time()
    auto_annotate(
        data=paths[2],
        det_model=YW[i],
        sam_model=SAM_model[0],
        device="cuda",
        output_dir=fr"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\Worldly_SAM\trials\fescue\YW_Mobile\{YW[i]}",
        prompt=prompt[2][i],
        confidence=conf[2][i],
        box_threshold=0.5
    )
    total = t.time()-start
    festimes.append(total)
    print(f"\n\nTime for {YW[i]}: {total}")


 YW with mobile on fescue
Results saved to runs\detect\predict1852
11 labels saved to runs\detect\predict1852\labels
Results saved to runs\segment\predict74
Results saved to runs\segment\predict74
Results saved to runs\segment\predict74
Results saved to runs\segment\predict74
Results saved to runs\segment\predict74
Results saved to runs\segment\predict74
Results saved to runs\segment\predict74
Results saved to runs\segment\predict74
Results saved to runs\segment\predict74
Results saved to runs\segment\predict74
Results saved to runs\segment\predict74


Time for yolov8s-worldv2.pt: 3.9544880390167236


Results saved to runs\detect\predict1853
11 labels saved to runs\detect\predict1853\labels
Results saved to runs\segment\predict75
Results saved to runs\segment\predict75
Results saved to runs\segment\predict75
Results saved to runs\segment\predict75
Results saved to runs\segment\predict75
Results saved to runs\segment\predict75
Results saved to runs\segment\predict75
Results saved to ru

In [ ]:
print("\n YW with mobile on buds")
for i in range(4):
    start = time.time()
    auto_annotate(
        data=paths[3],
        det_model=YW[i],
        sam_model=SAM_model[0],
        device="cuda",
        output_dir=fr"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\Worldly_SAM\trials\buds\YW_Mobile\{YW[i]}",
        prompt=prompt[3][i],
        confidence=conf[3][i],
        box_threshold=0.3
    )
    total = time.time()-start
    print(f"\n\nTime for {YW[i]}: {total}")
    budtimes.append(total)

## Base SAM

In [ ]:
print("YW with Base on berries")
for i in range(4):
    start = time.time()
    auto_annotate(
        data=paths[0],
        det_model=YW[i],
        sam_model=SAM_model[1],
        device="cuda",
        output_dir=fr"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\Worldly_SAM\trials\berries\YW_Base\{YW[i]}",
        prompt=prompt[0][i],
        confidence=conf[0][i],
        box_threshold=0.4
    )
    total = time.time()-start
    print(f"\n\nTime for {YW[i]}: {total}")
    bertimes.append(total)

In [ ]:
print("\n YW with Base on red leaf")
for i in range(4):
    start = time.time()
    auto_annotate(
        data=paths[1],
        det_model=YW[i],
        sam_model=SAM_model[1],
        device="cuda",
        output_dir=fr"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\Worldly_SAM\trials\red leaf\YW_Base\{YW[i]}",
        prompt=prompt[1][i],
        confidence=conf[1][i],
        box_threshold=0.5
    )
    total = time.time()-start
    print(f"\n\nTime for {YW[i]}: {total}")
    rltimes.append(total)

In [ ]:
print("\n YW with Base on fescue")
for i in range(4):
    start = t.time()
    auto_annotate(
        data=paths[2],
        det_model=YW[i],
        sam_model=SAM_model[1],
        device="cuda",
        output_dir=fr"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\Worldly_SAM\trials\fescue\YW_Base\{YW[i]}",
        prompt=prompt[2][i],
        confidence=conf[2][i],
        box_threshold=0.5
    )
    total = t.time()-start
    print(f"\n\nTime for {YW[i]}: {total}")
    festimes.append(total)

In [ ]:
print("\n YW with Base on buds")
for i in range(4):
    start = time.time()
    auto_annotate(
        data=paths[3],
        det_model=YW[i],
        sam_model=SAM_model[1],
        device="cuda",
        output_dir=fr"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\Worldly_SAM\trials\buds\YW_Base\{YW[i]}",
        prompt=prompt[3][i],
        confidence=conf[3][i],
        box_threshold=0.25
    )
    total = time.time()-start
    print(f"\n\nTime for {YW[i]}: {total}")
    budtimes.append(total)

## SAM Large

In [ ]:
print("YW with Large on berries")
for i in range(4):
    start = time.time()
    auto_annotate(
        data=paths[0],
        det_model=YW[i],
        sam_model=SAM_model[2],
        device="cuda",
        output_dir=fr"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\Worldly_SAM\trials\berries\YW_Large\{YW[i]}",
        prompt=prompt[0][i],
        confidence=conf[0][i],
        box_threshold=0.4
    )
    total = time.time() - start
    print(f"\n\nTime for {YW[i]}: {total}")
    bertimes.append(total)

In [ ]:
print("\n YW with Large on red leaf")
for i in range(4):
    start = time.time()
    auto_annotate(
        data=paths[1],
        det_model=YW[i],
        sam_model=SAM_model[2],
        device="cuda",
        output_dir=fr"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\Worldly_SAM\trials\red leaf\YW_Large\{YW[i]}",
        prompt=prompt[1][i],
        confidence=conf[1][i],
        box_threshold=0.5
    )
    total = time.time() - start
    print(f"\n\nTime for {YW[i]}: {total}")
    rltimes.append(total)

In [ ]:
print("\n YW with Large on fescue")
for i in range(4):
    start = t.time()
    auto_annotate(
        data=paths[2],
        det_model=YW[i],
        sam_model=SAM_model[2],
        device="cuda",
        output_dir=fr"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\Worldly_SAM\trials\fescue\YW_Large\{YW[i]}",
        prompt=prompt[2][i],
        confidence=conf[2][i],
        box_threshold=0.5
    )
    total = t.time() - start
    festimes.append(total)
    print(f"\n\nTime for {YW[i]}: {total}")

In [ ]:
print("\n YW with Large on buds")
for i in range(4):
    start = time.time()
    auto_annotate(
        data=paths[3],
        det_model=YW[i],
        sam_model=SAM_model[2],
        device="cuda",
        output_dir=fr"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\Worldly_SAM\trials\buds\YW_Large\{YW[i]}",
        prompt=prompt[3][i],
        confidence=conf[3][i],
        box_threshold=0.25
    )
    total = time.time() - start
    print(f"\n\nTime for {YW[i]}: {total}")
    budtimes.append(total)

In [ ]:
for time in bertimes:
    print(time)

In [ ]:
for time in rltimes:
    print(time)

In [ ]:
for time in budtimes:
    print(time)

In [ ]:
for time in festimes:
    print(time)

# EfficientSAM

In [ ]:
from efficient_sam.build_efficient_sam import build_efficient_sam_vitt, build_efficient_sam_vits
from torchvision import transforms

esam_models = [build_efficient_sam_vitt(), build_efficient_sam_vits()]

In [16]:
from efficient_sam.build_efficient_sam import build_efficient_sam_vitt, build_efficient_sam_vits
from torchvision import transforms

ground_truth_paths = [r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\berries-1\train\images",
                      r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\red-leaf-1\train\images",
                      r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\fescue-1\train\images",
                      r"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\buds-1\train\images"]
dino_models = ['swint', 'swinb']
YW = ['yolov8s-worldv2.pt', 'yolov8m-worldv2.pt', 'yolov8l-worldv2.pt', 'yolov8x-worldv2.pt']
prompts = [
    ['a blue fruit','spherical blueberry','a single, round blueberry','a blue fruit'],
    ['Bright red leaves contrasting with surrounding green leaves','a cluster of red leaves among green plants','a cluster of red leaves among green vegetation','A cluster of red leaves surrounded by green foliage'],
    ['Fescue grass spots','Regions of fescue grass','Fescue patches','Clusters of grass'],
    ['Bud with leaves','Bud with leaves','Bud with sprouting leaves','Bud with leaves']
]
confidences = [[0.008,0.0012,0.0001,0.0115],[0.037,0.01,0.0001,0.01],[0.0004,0.0001,0.0006,0.0023],[0.0012,0.007,0.0002,0.0028]]
esam_models = [build_efficient_sam_vitt(), build_efficient_sam_vits()]
sam_models = ['efficientsam_ti', 'efficientsam_s']
categories = ['berries', 'red leaf', 'fescue', 'buds']
max_areas = [0.4, 0.4, 0.7, 0.25]
folders = {'berries': 'berries-1', 'red leaf': 'red-leaf-1', 'fescue': 'fescue-1', 'buds': 'buds-1'}

start = t.time()
for x in range(min(len(categories),1)):
    for z in range(len(sam_models)):
        img_path = ground_truth_paths[x]

        prompt = prompts[x][1]
        conf = confidences[1][x]
        start = t.time()
        for fname in os.listdir(img_path):
            path = img_path + "\\" + fname
            results = run_yolo_world(path, prompt, conf, 'yolov8l-worldv2.pt')
            boxes = results[0].boxes.xyxy.cpu().tolist()
            print([([(x[0]+((x[2]-x[0]))/2),(x[1]+((x[3]-x[1]))/2)]) for x in boxes])
            print(boxes)
            #print(len(boxes))
            #print(f'conf{conf}, prompt{prompt}')
            image_np = np.array(Image.open(path))
            image_tensor = transforms.ToTensor()(image_np)
            input_points = boxes
            input_points = torch.tensor([[[[100,200]]]])
            input_labels = torch.tensor([[[1]]])

            predicted_logits, predicted_iou = esam_models[z](
                image_tensor[None, ...],
                input_points,
                input_labels,
            )
            mask = torch.ge(predicted_logits[0, 0, 0, :, :], 0).cpu().detach().numpy()
            print(mask)
            #sam_results = model(os.path.join(img_path, fname), model= model, bboxes=boxes, verbose=False)
        print(f"\n\nTime for {categories[x]},YW_l,{sam_models[z]}: {t.time() - start}")


image 1/1 C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\berries-1\train\images\IMG_9331_jpg.rf.72224121a63b7fbdd24a1ec1c8e82326.jpg: 448x640 7 spherical blueberrys, 20.0ms
Speed: 2.0ms preprocess, 20.0ms inference, 2.0ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs\detect\predict124
1 label saved to runs\detect\predict124\labels
[[1005.9503784179688, 723.2714233398438, 1117.0848388671875, 873.8399658203125], [1012.791015625, 725.1598510742188, 1088.3660888671875, 799.3184814453125], [791.8519897460938, 537.2545776367188, 835.5354614257812, 598.3583374023438], [752.61669921875, 1018.3922119140625, 826.1536254882812, 1093.3017578125], [818.8764038085938, 1009.9738159179688, 879.9220581054688, 1074.2222900390625], [789.5413208007812, 535.2908325195312, 877.39111328125, 633.9640502929688], [1710.5672607421875, 988.0695190429688, 1869.229736328125, 1177.4019775390625]]
[[1061.5176086425781, 798.5556945800781], [1050.5785522460938, 762.2

# Segmentation Evaluation

In [ ]:
sam_models = ['YW_Base','YW_Mobile','YW_Large']
categories = ['berries','fescue','red leaf','buds']
folders = {'berries':'berries-1', 'red leaf':'red-leaf-1','fescue':'tiled-fescue-seg-1','buds':'buds-1'}

for category in categories:
    for sam_model in sam_models:
        for yolo_model in YW:
            metrics = process_files_seg(fr"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\Worldly_SAM\trials\{category}\{sam_model}\{yolo_model}", fr'C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\{folders[category]}\train\labels')
            for score in metrics['iou_scores']:
                print(score)
            print(rf"{category}\{yolo_model}\{sam_model}: Mean IOU = {np.mean(metrics['iou_scores'])}")

In [ ]:
sam_models = ['YW_Base','YW_Mobile','YW_Large']
categories = ['fescue']
folders = {'fescue':'tiled-fescue-seg-1'}

for category in categories:
    for sam_model in sam_models:
        for yolo_model in YW:
            metrics = process_files_seg(fr"C:\Users\Mechanized Systems\DataspellProjects\WSU_joint_data\Auto Annotate\Worldly_SAM\trials\{category}\{sam_model}\{yolo_model}", fr'C:\Users\Mechanized Systems\DataspellProjects\AutoAnnotate\autoannotate study\tiled-fescue-seg-1\train\labels')
            for score in metrics['iou_scores']:
                print(score)
            print(rf"{category}\{yolo_model}\{sam_model}: Mean IOU = {np.mean(metrics['iou_scores'])}")